# Define Functions

In [158]:
def find_max(col, DATA_COUNT):
    max_num = -999
    for i in range(DATA_COUNT):
        if col[i] > max_num:
            max_num = col[i]
    return max_num

def find_max_pos(col, DATA_COUNT):
    max_num = -999
    pos = -999
    for i in range(DATA_COUNT):
        if col[i] > max_num:
            max_num = col[i]
            pos = i
    return max_num, pos

def find_min(col, DATA_COUNT):
    min_num = 100000
    for i in range(DATA_COUNT):
        if col[i] < min_num:
            min_num = col[i]
    return min_num

def unique_type_count(col_array):
    
    toggle_0=0
    toggle_1=0
    toggle_2=0
    toggle_3=0
    toggle_4=0
    toggle_5=0
    toggle_6=0
    toggle_7=0
    toggle_8=0
    toggle_9=0
    
    for i in range(len(col_array)):
        if col_array[i] == 0: toggle_0=1
        if col_array[i] == 1: toggle_1=1
        if col_array[i] == 2: toggle_2=1
        if col_array[i] == 3: toggle_3=1
        if col_array[i] == 4: toggle_4=1
        if col_array[i] == 5: toggle_5=1
        if col_array[i] == 6: toggle_6=1
        if col_array[i] == 7: toggle_7=1
        if col_array[i] == 8: toggle_8=1
        if col_array[i] == 9: toggle_9=1
            
    num_unique=toggle_0+toggle_1+toggle_2+toggle_3+toggle_4+toggle_5+toggle_6+toggle_7+toggle_8+toggle_9
    return num_unique

# Read Data

In [17]:
f = open("../glass.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

_attr, attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9 = [], [], [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[1]))   
    attr_1.append(float(row_list[2]))  
    attr_2.append(float(row_list[3]))  
    attr_3.append(float(row_list[4]))
    attr_4.append(float(row_list[5]))
    attr_5.append(float(row_list[6]))  
    attr_6.append(float(row_list[7]))  
    attr_7.append(float(row_list[8]))
    attr_8.append(float(row_list[9]))
    attr_9.append(row_list[10]) # 這個是 class
    

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有214個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8, attr_9]

# Equal-Width Discretization

In [31]:
Attributes_EWD = []
for i in range(10):
    
    specific_col = Attributes[i]
    
    if i != 9:
        number_of_intervals = 10
        max_num_in_col = find_max(specific_col, DATA_COUNT)
        min_num_in_col = find_min(specific_col, DATA_COUNT)
        width = (max_num_in_col - min_num_in_col)/number_of_intervals

        # Decide 9 Split points, so that 10 intervals will form
        split_point_1 = max_num_in_col - 1 * width
        split_point_2 = max_num_in_col - 2 * width
        split_point_3 = max_num_in_col - 3 * width
        split_point_4 = max_num_in_col - 4 * width
        split_point_5 = max_num_in_col - 5 * width
        split_point_6 = max_num_in_col - 6 * width
        split_point_7 = max_num_in_col - 7 * width
        split_point_8 = max_num_in_col - 8 * width
        split_point_9 = max_num_in_col - 9 * width

        # transfer attributes into proper interval
        discrete_attr = []
        for i in range(DATA_COUNT):
            discrete_attr.append(-100)

        for i in range(DATA_COUNT):

            num = specific_col[i]

            if (num >= split_point_1): discrete_attr[i] = 0
            elif (num >= split_point_2) and (num < split_point_1): discrete_attr[i] = 1
            elif (num >= split_point_3) and (num < split_point_2): discrete_attr[i] = 2
            elif (num >= split_point_4) and (num < split_point_3): discrete_attr[i] = 3
            elif (num >= split_point_5) and (num < split_point_4): discrete_attr[i] = 4
            elif (num >= split_point_6) and (num < split_point_5): discrete_attr[i] = 5
            elif (num >= split_point_7) and (num < split_point_6): discrete_attr[i] = 6
            elif (num >= split_point_8) and (num < split_point_7): discrete_attr[i] = 7
            elif (num >= split_point_9) and (num < split_point_8): discrete_attr[i] = 8
            elif (num  < split_point_9): discrete_attr[i] = 9
                
        Attributes_EWD.append(discrete_attr)
    else:
        Attributes_EWD.append(specific_col)

# Selective Naïve Bayes (w. Laplace estimate)

In [165]:
Y_col = Attributes_EWD[9]


# 假設我現在挑到 2 個 column ==> 2, [0, 1]
col_name = [3, 7]
col_num = len(col_name)

DATUM = []

for i in range(DATA_COUNT):
    
    data = []    
    for j in range(col_num):
        data.append(Attributes_EWD[col_name[j]][i])

    DATUM.append(data)

# 這個 loop 是在區分 testing data 和 training data 用的
for i in range(DATA_COUNT):
    
    # 區分 training data 和 testing data
    X_test = DATUM[i]
    Y_test = Y_col[i]
        
    X_train = []
    Y_train = []
    for j in range(DATA_COUNT):
        if j != i:
            X_train.append(DATUM[j]) # 照理來說應該只能有 213 筆
            Y_train.append(Y_col[j]) # 照理來說應該只能有 213 筆
    
    # 在 i-th RUN 裡面
    # 目標是拿著 X_test 然後到 X_train 裡面算 Naive Bayes with Laplace Esitmate 的 score，然後再用 y_test 確認有無正確
    # 有 6 種 class type: ['1\n', '2\n', '3\n', '5\n', '6\n', '7\n']
    
    # (1) 計算 p(c), 

    count_c1 = 0
    count_c2 = 0
    count_c3 = 0
    count_c5 = 0
    count_c6 = 0
    count_c7 = 0
    
    for j in range(DATA_COUNT-1):
        
        if   (Y_train[j] == "1\n"): # 目的：計算 p(c=c1)，意義：Given c=c1
            count_c1=count_c1+1
            
        elif (Y_train[j] == "2\n"): # 目的：計算 p(c=c2)，意義：Given c=c2
            count_c2=count_c2+1
            
        elif (Y_train[j] == "3\n"): # 目的：計算 p(c=c3)，意義：Given c=c3
            count_c3=count_c3+1
            
        elif (Y_train[j] == "5\n"): # 目的：計算 p(c=c5)，意義：Given c=c5
            count_c5=count_c5+1
            
        elif (Y_train[j] == "6\n"): # 目的：計算 p(c=c6)，意義：Given c=c6
            count_c6=count_c6+1
            
        elif (Y_train[j] == "7\n"): # 目的：計算 p(c=c7)，意義：Given c=c7
            count_c7=count_c7+1
        
    prob_c1 = count_c1/(DATA_COUNT - 1)
    prob_c2 = count_c2/(DATA_COUNT - 1)
    prob_c3 = count_c3/(DATA_COUNT - 1)
    prob_c5 = count_c5/(DATA_COUNT - 1)
    prob_c6 = count_c6/(DATA_COUNT - 1)
    prob_c7 = count_c7/(DATA_COUNT - 1)
    
    # (2) 計算 p(x,c)
    cond_prob_c1=1
    cond_prob_c2=1
    cond_prob_c3=1
    cond_prob_c5=1
    cond_prob_c6=1
    cond_prob_c7=1
        
    for k in range(col_num):
        target_val = X_test[k]
        target_count_c1 = 0
        target_count_c2 = 0
        target_count_c3 = 0
        target_count_c5 = 0
        target_count_c6 = 0
        target_count_c7 = 0
        
        # 其實原本想要用 Attributes_EWD[Col_num]，但是後來發現我的 training dataset 是 DATA_COUNT - 1，而非 DATA_COUNT
        # 而在檢查過後確認，有的類別確實只有一個 sample，所以才不照原本的寫法
        target_col=[]
        for j in range(DATA_COUNT-1):
            target_col.append(X_train[j][k])
            
        for j in range(DATA_COUNT-1):

            if   (Y_train[j] == "1\n") and (target_val == X_train[j][k]): 
                target_count_c1=target_count_c1+1

            elif (Y_train[j] == "2\n") and (target_val == X_train[j][k]):
                target_count_c2=target_count_c2+1

            elif (Y_train[j] == "3\n") and (target_val == X_train[j][k]):
                target_count_c3=target_count_c3+1

            elif (Y_train[j] == "5\n") and (target_val == X_train[j][k]):
                target_count_c5=target_count_c5+1

            elif (Y_train[j] == "6\n") and (target_val == X_train[j][k]):
                target_count_c6=target_count_c6+1

            elif (Y_train[j] == "7\n") and (target_val == X_train[j][k]):
                target_count_c7=target_count_c7+1
        
        # (2.5) 考慮 Laplace Estimate
        cond_prob_c1=cond_prob_c1*(target_count_c1+1)/(count_c1+unique_type_count(target_col))
        cond_prob_c2=cond_prob_c2*(target_count_c2+1)/(count_c2+unique_type_count(target_col))
        cond_prob_c3=cond_prob_c3*(target_count_c3+1)/(count_c3+unique_type_count(target_col))
        cond_prob_c5=cond_prob_c5*(target_count_c5+1)/(count_c5+unique_type_count(target_col))
        cond_prob_c6=cond_prob_c6*(target_count_c6+1)/(count_c6+unique_type_count(target_col))
        cond_prob_c7=cond_prob_c7*(target_count_c7+1)/(count_c7+unique_type_count(target_col))

    # (3) 比較 6 個 score 看誰最高
    score_c1=prob_c1*cond_prob_c1
    score_c2=prob_c2*cond_prob_c2
    score_c3=prob_c3*cond_prob_c3
    score_c5=prob_c5*cond_prob_c5
    score_c6=prob_c6*cond_prob_c6
    score_c7=prob_c7*cond_prob_c7
    score_list = [score_c1, score_c2, score_c3, score_c5, score_c6, score_c7]
    
    # (4) 看看最高的那個類別有沒有和 y_test 一致
    print(find_max_pos(score_list,6), Y_test)

(0.08451398841842771, 0) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.08451398841842771, 0) 1

(0.0563068332475473, 1) 1

(0.08451398841842771, 0) 1

(0.08451398841842771, 0) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.08451398841842771, 0) 1

(0.08451398841842771, 0) 1

(0.04041973359142195, 0) 1

(0.08451398841842771, 0) 1

(0.0563068332475473, 1) 1

(0.13889018867728334, 1) 1

(0.014698084942335253, 0) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.08451398841842771, 0) 1

(0.08451398841842771, 0) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.08451398841842771, 0) 1

(0.08451398841842771, 0) 1

(0.13889018867728334, 1) 1

(0.13889018867728334, 1) 1

(0.08451398841842771,

In [161]:
print(find_max_pos(score_list,6))
score_list

(0.04984158291343172, 0)


[0.04984158291343172,
 0.03378409994852838,
 0.003192488262910798,
 0.0008960217165973441,
 0.0014535094986845736,
 0.0007746812787583729]